# Regular Expressions - Codealong

## Introduction

In this lab, we'll make use of some common regex patterns to search through text. 

## Objectives

In this lab you will: 

- Create regex code to capture meaningful patterns found in text 

## Getting Started

In this codealong, we're going to explore some of the more common operations used to filter text and match patterns with regular expressions, or _regex_ for short. 


## Using Regex Testers

Working with regex patterns is an iterative process -- it's quite rare that we'll write the pattern that does exactly what we want while capturing all edge cases the first time through. To make this process simpler, most developers and data scientists make use of popular regex tester websites, which allow you to put in a block of text and test your patterns by quickly seeing visually what a pattern will grab out of the text block. During this codealong, we strongly recommend you take some time to use a regex tester website such as [Regex Pal](https://www.regexpal.com/) or [regexr](https://regexr.com/) so that you can visually inspect how changing your regex pattern affects your results when working towards a correct answer! 

## Regex Cheat Sheet

When learning regex, the symbols and notation for patterns can seem a bit intimidating. Don't worry about trying to memorize these symbols and patterns -- it's easier to just keep a cheat sheet handy and look them up as needed. Instead, focus on understanding the basic structure of regex patterns, and then look up the syntax needed to build your pattern.  

For reference, we've embedded a common Regex Cheat Sheet found online. Core regex syntax tends to be the same across all languages, so you don't have to worry too much about language-specific features for regex, or looking up regex for the wrong language. Although some languages may add in some special bells and whistles for regex, the core functionality and syntax for regex is generally the same across the board. 

<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/regex_cheat_sheet.png'>


### The Data

We'll be working with a plain text file in this repo called `'menu.txt'`. Our goal will be to use regex to scrape various types of information from this file.

Run the cell below to import everything we'll need, as well as loading in the file itself. 

In [17]:
# using regex to scrape various types of information from this file
import re

with open('menu.txt', 'r') as f:
    file = f.read()
    
print(file)

Flatiron School Cafe Menu


Appetizers

Nachos - $10
Calamari - $12
3 Cheese Platter - $8.75

Entrees

Chicken Sandwich - $16.95
A fried chicken sandwich with lettuce, tomato, and mayo. Add cheese for $1.50

Flatiron Steak - $22
A prime cut of Flatiron Steak, cooked to your liking. Comes with a side of vegetables. Add a salad or cup of soup for $3

Garden Salad - $14
A salad with stuff from the garden on our roof. 3 types of dressing available.


Want to place an order for delivery? Call us at (555) 123-8452!



In [ ]:
# import re

# with open('menu.txt', 'r') as f:
#     file = f.read()

# print(file)

### Creating a Basic Pattern With Character Classes

We'll start by creating a basic pattern, and then using the `re` library to search through the file for any instances that match this pattern. 

The most simple types of patterns we can use are character classes. These allow us to match predefined things such as words or digits. Let's quickly try getting all the digits. 

Run the cell below to find all the digits in the document. 

In [3]:
pattern = '\d' # the pattern '\d' regular expression that matches any single digit (0 - 9)
p = re.compile(pattern) # re.compile() function compiles the regular expression into regex object
digits = p.findall(file) # use the .findall() finds all the overlapping matches of the patterns in the text
digits


['1',
 '0',
 '1',
 '2',
 '3',
 '8',
 '7',
 '5',
 '1',
 '6',
 '9',
 '5',
 '1',
 '5',
 '0',
 '2',
 '2',
 '3',
 '1',
 '4',
 '3',
 '5',
 '5',
 '5',
 '1',
 '2',
 '3',
 '8',
 '4',
 '5',
 '2']

In [ ]:
# pattern = '\d'
# p = re.compile(pattern)
# digits = p.findall(file)
# digits

It worked, but not in the way we might have expected. This is because the pattern has found every individual number as an individual match. If you look at the order of the numbers and compare it to the menu text above, you notice that they are in the order you'd see if you read them from left to right starting at the top of the menu. 

Let's try modifying our pattern, so that it only gets numbers with a dollar sign in front of them. 

#### Escaping Metacharacters

In regex, the dollar sign is a **_Metacharacter_**. This character means something. We can't just use the dollar sign in a pattern, anymore that we can use a reserved keyword like `for` as a variable name in Python. Since the dollar sign is a reserved symbol in regex, we'll need to escape it using a `\`.  This tells the regex compiler that we are talking about the actual dollar sign symbol, not using it to talk about the end of a string. 
In the cell below:

* Modify the pattern so that it includes a dollar sign, followed by a digit character class  
* Compile the new pattern  
* Use the compiled pattern's `.findall()` method on the pattern  
* Display the results

**_NOTE:_** Make sure there is no space between the dollar sign and the digit character class, since it will treat the space as a character that is a part of the pattern! 

In [8]:
# lets modify the pattern such that it includes the dollar sign followed by the digit character
pattern = '\$\d'
p = re.compile(pattern)
digits = p.findall(file)

print(digits)


['$1', '$1', '$8', '$1', '$1', '$2', '$3', '$1']


In [4]:
# pattern = '\$\d'
# p = re.compile(pattern)
# digits = p.findall(file)
# digits

['$1', '$1', '$8', '$1', '$1', '$2', '$3', '$1']

That's better, but still not perfect! Now, the pattern only gets the first digit for prices. For instance, the pattern truncates `$10` to `$1`.

To fix this, we could also get the next 3 characters that follow a match by adding `.{3}` to the end of pattern. Let's try this in the cell below.  

In [10]:
# for the above code the pattern truncates USD 10 to USD 1.
# to fix lets get the 3 characters that matches the pattern {3}

pattern = "\$\d.{3}"
p = re.compile(pattern)
digits = p.findall(file)
print(digits)

['$8.75', '$16.9', '$1.50']


In [11]:
# pattern = '\$\d.{3}'
# p = re.compile(pattern)
# digits = p.findall(file)
# digits

This seems to have worked for some, but not all. It also left out any prices have less than 3 characters after the initial match, such as \$10. 


We need to create a pattern that gets all the numbers in a price. To do this, we'll make use of groups and ranges!

### Using Groups, Ranges, and Quantifiers

Groups and ranges allow us to create more complex patterns by grouping them together. In these groups, we can also provide rangers, to make our patterns less verbose without making them any less powerful. If we want to get any lowercase letters, we could create a group that contains a letter range like `[a-z]`. If we wanted to get all letters regardless of case, we could use the range `[a-zA-Z]`. We could even include things like numbers in here. If wanted to create a grouping that matches any numbers 0-9, or any uppercase or lowercase letters between a and z, we could use `[a-zA-z0-9]`. 

Run this pattern in the cell below, and see what it matches. 

In [12]:
# group and ranges allow to create more complex patterns
# '[a-z]' creates a group that contains lower case letters
# [A - Z] for upper case letters
# [a-zA-Z] gets the letters regardless of the case
# [a-zA-Z0-9] creates grouping the matches any numbers 0-9, uppercase and lowercase

pattern = '[a-zA-Z0-9]'
p = re.compile(pattern)
digits = p.findall(file)
print(digits)


['F', 'l', 'a', 't', 'i', 'r', 'o', 'n', 'S', 'c', 'h', 'o', 'o', 'l', 'C', 'a', 'f', 'e', 'M', 'e', 'n', 'u', 'A', 'p', 'p', 'e', 't', 'i', 'z', 'e', 'r', 's', 'N', 'a', 'c', 'h', 'o', 's', '1', '0', 'C', 'a', 'l', 'a', 'm', 'a', 'r', 'i', '1', '2', '3', 'C', 'h', 'e', 'e', 's', 'e', 'P', 'l', 'a', 't', 't', 'e', 'r', '8', '7', '5', 'E', 'n', 't', 'r', 'e', 'e', 's', 'C', 'h', 'i', 'c', 'k', 'e', 'n', 'S', 'a', 'n', 'd', 'w', 'i', 'c', 'h', '1', '6', '9', '5', 'A', 'f', 'r', 'i', 'e', 'd', 'c', 'h', 'i', 'c', 'k', 'e', 'n', 's', 'a', 'n', 'd', 'w', 'i', 'c', 'h', 'w', 'i', 't', 'h', 'l', 'e', 't', 't', 'u', 'c', 'e', 't', 'o', 'm', 'a', 't', 'o', 'a', 'n', 'd', 'm', 'a', 'y', 'o', 'A', 'd', 'd', 'c', 'h', 'e', 'e', 's', 'e', 'f', 'o', 'r', '1', '5', '0', 'F', 'l', 'a', 't', 'i', 'r', 'o', 'n', 'S', 't', 'e', 'a', 'k', '2', '2', 'A', 'p', 'r', 'i', 'm', 'e', 'c', 'u', 't', 'o', 'f', 'F', 'l', 'a', 't', 'i', 'r', 'o', 'n', 'S', 't', 'e', 'a', 'k', 'c', 'o', 'o', 'k', 'e', 'd', 't', 'o',

In [15]:
# pattern = '[a-zA-Z0-9]'
# p = re.compile(pattern)
# digits = p.findall(file)
# digits

Now, let's use some ranges, groupings, and quantifiers to match on any price in the menu. Write this pattern now in the cell below. Be sure to include the dollar signs in the pattern to match!

**_NOTE_**: This one will probably take you a little while to figure out. Take some time to study all the potential cases we want to match inside the menu. Also, don't forget to escape metacharacters such as dollar signs and decimal points!

In [16]:
# creating a pattern designed to match the monetary values that start with the $ (dollar sign)
pattern = '(\$\d+\.?\d*)' 
p = re.compile(pattern)
digits = p.findall(file)
print(digits)

['$10', '$12', '$8.75', '$16.95', '$1.50', '$22', '$3', '$14']


In [ ]:
# pattern = '(\$\d+\.?\d*)'

# The pattern is a regular expression designed to match monetary values that start with a dollar sign ($). Lets break it down:
# \$: Matches the literal dollar sign ($). The backslash is required to escape it because $ has a special meaning in regex (indicates the end of a string).
# \d+: Matches one or more digits (0-9). This captures the whole number part of the dollar amount (e.g., $123).
# \.?: Matches an optional decimal point (.). The ? means the decimal point may or may not be present.
# \d*: Matches zero or more digits following the decimal point. This allows for amounts like $123.45 or just $123.

### Putting It All Together

To end this lab, we'll have you put your newfound regex skills to the test to see if you can write a pattern that gets the phone number from the bottom of the menu. How you match it is up to you, just as long as you get the phone number! It's okay if the dashes and parentheses are included in your match, but not the exclamation point at the end. 

In the cell below, write a regex pattern to match the phone number from the menu, and confirm that it works by compiling it and running in it. 

In [22]:
# Want to place an order for delivery? Call us at (555) 123-8452
# lets write a pattern that gets the phone number from the bottom of the menu
pattern = '(\(\d{3}\) (\d{3}-\d{4}))'
p = re.compile(pattern)
final_digits = p.findall(file)
print(final_digits)

[('(555) 123-8452', '123-8452')]


## Summary

In this lab, we got some practice in using regex to filter text!